In [32]:
import os
from pathlib import Path
# from matplotlib import pyplot as plt
# from matplotlib import cm
import pandas as pd
import yaml
# import glob
# import seaborn as sns
# from Bio import SeqIO

In [33]:
proj_dir="/master/nplatt/sch_hae_its-nigeria"
results_dir=f"{proj_dir}/results"

In [34]:
info_df=pd.read_csv(f"{proj_dir}/its-nigeria_samplesheet.csv", sep=",")
info_df

,wgs_id,its_id,sra,species,country,locale,patient,miracidum
0,Sb_NG_ak_1.1,Sb.ng.ak.1.1F,NaN,sbovis,nigeria,ak,1,1
1,Sb_NG_ak_2.1,sb_ng_ak_2.1,NaN,sbovis,nigeria,ak,2,1
2,Sb_NG_ak_2.2,Sb.ng.ak.2.2F,NaN,sbovis,nigeria,ak,2,2
3,Sb_NG_ak_2.3,Sb.ng.ak.2.3F,NaN,sbovis,nigeria,ak,2,3
4,Sb_NG_ak_3.1,Sb.ng.ak.3.1R,NaN,sbovis,nigeria,ak,3,1
...,...,...,...,...,...,...,...,...
200,Sh_NG_os_3_1,sh_ng_os_3_1,NaN,shaematobium,nigeria,osun,3,1
201,c_Sh_NG_os_3_11,Sh.ng.os.3.11F,NaN,shaematobium,nigeria,osun,3,11
202,c_Sh_NG_os_3_5,Sh.ng.os.3.5F,NaN,shaematobium,nigeria,osun,3,5
203,c_Sh_NG_os_3_6,Sh.ng.os.3.6F,NaN,shaematobium,nigeria,osun,3,6


# Whole Mitochondria phylogeny

In [35]:
Path(f"{results_dir}/mito_phylogeny").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/mito_phylogeny")

In [ ]:
%%bash

conda run -n mito_assembly --live-stream clustalo \
    -INFILE=../mito_assembly/mito_assemblies.fasta \
    -OUTFILE=mito_assemblies.aligned.fasta \
    -OUTPUT=FASTA

In [36]:
%%bash

conda run -n trimal trimal -in mito_assemblies.aligned.fasta -out mito_assemblies.aligned.trimmed.fasta -automated1

In [ ]:
%%bash 

conda run -n raxml --live-stream \
    raxml-ng \
        --all \
        --msa mito_assemblies.aligned.trimmed.fasta \
        --msa-format FASTA \
        --model GTR \
        --tree pars{50},rand{50} \
        --bs-trees 1000 \
        --redo


RAxML-NG v. 1.2.2 released on 11.04.2024 by The Exelixis Lab.
exandros Stamatakis. M. Kozlov and Al
ettisworth, Julia Haag, Anastasis Togkousidis.oit Morel, Sarah Lutteropp, Ben B
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 6262V CPU @ 1.90GHz, 96 cores, 1007 GB RAM (allocated: 6 physical cores)

RAxML-NG was called at 10-Feb-2026 14:22:16 as follows:

d.trimmed.fasta --msa-format FASTA --model GTR --tree pars{50},rand{50} --bs-trees 1000 --redo

ysis options:
  run mode: ML tree search + bootstrapping (Felsenstein Bootstrap)
ndom (50) + parsimony (50)
  bootstrap replicates: parsimony (1000)
  random seed: 1770754936
inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
eral: 10.000000, brlen-triplet: 1000.000000
  fast spr radius: AUTO
  spr subtree cutoff: 1.000000
 fast CLV updates: ON
  branch lengths: proportional (ML estim

In [ ]:
%%bash
conda run -n nwed nw_ed mito_assemblies.aligned.trimmed.fasta.raxml.support 'i & b<=50' o > mito_assemblies.aligned.trimmed.fasta.raxml.support.bs50.nwk

In [ ]:
%%bash
cat ../combined/merged.csv | sed 's/,/\t/g' | cut -f1,3,4,5,17,19,22 >annotations.tsv

# Cox1 phylogeny

In [26]:
Path(f"{results_dir}/cox1_phylogeny").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/cox1_phylogeny")

In [ ]:
%%bash

#get reference mito >reference_mito.fas
conda run -n entrez efetch -db nuccore -id PP905348.1 -format fasta > PP905348.1.fasta

cat ../mito_phylogeny/mito_assemblies.aligned.trimmed.fasta PP905348.1.fasta >mito_cox1.fas

In [28]:
%%bash

# I gave this an initial trim to make the alignment faster (manually)
conda run -n mito_assembly --live-stream clustalo \
    --dealign \
    -INFILE=mito_cox1.pre-trimmed.fas \
    -OUTFILE=mito_cox1.aligned.fasta \
    -OUTPUT=FASTA

FORCED DEBUG: Potential Problem: sequences (N=204) don't have same lengths but contain gaps, consider using --dealign


Using 12 threads
Read 204 sequences (type: DNA) from mito_cox1.pre-trimmed.fas
igned input sequences as requested.
eqs) for mBed (from a total of 204 sequences)rom length sorted s
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 42.22u 0.03s 00:00:42.25 Elapsed: 00:00:20
 created 4 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 31.21u 0.00s 00:00:31.21 Elapsed: 00:00:14
Guide-tree computation (mBed) done.
Progressive alignment progress done. CPU time: 60.60u 1.19s 00:01:01.79 Elapsed: 00:00:52
Alignment written to mito_cox1.aligned.fasta


In [ ]:
# manually trimmed alignment to cox1 seqeunce. replace missing sequence at beginning and ends with ?

In [29]:
%%bash 

conda run -n raxml --live-stream \
    raxml-ng \
        --all \
        --msa cox1.trimmed.fasta \
        --msa-format FASTA \
        --model GTR \
        --tree pars{100},rand{100} \
        --bs-trees 1000 \
        --redo


RAxML-NG v. 1.2.2 released on 11.04.2024 by The Exelixis Lab.
exandros Stamatakis. M. Kozlov and Al
ettisworth, Julia Haag, Anastasis Togkousidis.oit Morel, Sarah Lutteropp, Ben B
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 6262V CPU @ 1.90GHz, 96 cores, 1007 GB RAM (allocated: 6 physical cores)

RAxML-NG was called at 10-Feb-2026 14:01:41 as follows:

sa-format FASTA --model GTR --tree pars{100},rand{100} --bs-trees 1000 --redo

Analysis options:
un mode: ML tree search + bootstrapping (Felsenstein Bootstrap)
imony (100)e(s): random (100) + pars
  bootstrap replicates: parsimony (1000)
  random seed: 1770753701
  tip-inner: OFF
ttern compression: ON
  per-rate scalers: OFF
  site repeats: ON
, brlen-triplet: 1000.000000.000000
  fast spr radius: AUTO
  spr subtree cutoff: 1.000000
es: ON CLV updat
  branch lengths: proportional (ML estimate, algorithm: N

In [30]:
%%bash
conda run -n nwed nw_ed cox1.trimmed.fasta.raxml.support 'i & b<=50' o > cox1.trimmed.fasta.raxml.support.bs50.nwk

In [31]:
%%bash
cat ../combined/merged.csv | sed 's/,/\t/g' | cut -f1,3,4,5,17,19,22 >annotations.tsv